Notebook này là source code dùng để thu thập tất cả link của các loài thú trên trang https://animalia.bio/

In [1]:
#Cài đặt selenium, chrome, chromedriver
#!pip install selenium
#!apt-get update 
#!apt install chromium-chromedriver
#!cp /usr/lib/chromium-browser/chromedriver /usr/bin
#Cài đặt beautifulsoup để crawl dữ liệu, vì beautifulsoup sẽ nhanh hơn selenium
#!pip install beautifulsoup4

In [2]:
import sys
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd

In [3]:
#Khởi tạo driver
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',options=chrome_options)


In [4]:
#Get url của trang cần lấy dữ liệu
driver.get("https://animalia.bio/mammals?page=1")

In [5]:
page = 1
while page < 151:
  try:
    driver.find_element("xpath", '/html/body/div[3]/div[2]/div/div[5]/div[3]/div[1]/span').click()
    page += 1
  except:
    pass

Cơ chế của website này là click vào button "LOAD MORE" thì sẽ cập nhật thêm dữ liệu của trang mới vào và dữ liệu trang cũ vẫn còn đó. Như vậy, ta tính được thú thì có 151 trang (nhóm chỉ xét lớp thú), nên bấm vào button đó 150 lần để load dữ liệu vào rồi lưu xuống file html để crawl dữ liệu một lần. Mặc dù khi bấm vào button "LOAD MORE" thì url của trang có đổi nhưng nhóm vẫn không hiểu vì sao nếu nhập tay url vào thì nó tự reset về trang 1. Vì vậy nhóm phải dùng selenium để click vào button 150 lần.
Test ở link :https://animalia.bio/mammals?page=1

In [6]:
#Lưu kết quả xuống file html
with open("animals.html", "w") as f:
    f.write(driver.page_source)

In [7]:
with open("animals.html") as fp:
    soup = BeautifulSoup(fp, "html.parser")
animals_urls = soup.find_all("a", {"class": "animals-invert__item"}) #Tìm tất cả các link của thú rồi lưu vào biến

In [8]:
animals_urls = [a['href'][:-11] for a in animals_urls] #Lấy link của từng thú, bỏ phần "?category=1"
animals_urls_df = pd.DataFrame(data = animals_urls, columns = ["url"])
animals_urls_df.to_csv('animals_urls.csv') #Xuất kết quả ra file csv